In [117]:
import requests 
import json
import pprint as pp # pretty print

import aries_cloudagent.wallet.crypto as crypto
import base64
import base58
import uuid
import hashlib
import jupyter_util as util
from importlib import reload
reload(util)

<module 'jupyter_util' from '/home/sevni/Documents/thclab/aries/aries-acapy-plugin/controller_acapy/jupyter_util.py'>

## Enter Invite Url:

In [118]:
agent1 = {}
plugin = {}
endpoint = "http://localhost:8150/connections/create-admin-invitation-url"
adminInvitationResponse = requests.post(endpoint)
adminInvitationResponse = adminInvitationResponse.text
agent1['url'] = json.loads(adminInvitationResponse)['invitation_url']
print(agent1['url'])

http://agent1.localhost?c_i=eyJAdHlwZSI6ICJkaWQ6c292OkJ6Q2JzTlloTXJqSGlxWkRUVUFTSGc7c3BlYy9jb25uZWN0aW9ucy8xLjAvaW52aXRhdGlvbiIsICJAaWQiOiAiYTBiNWY2NWMtNGZhYi00N2IwLThkY2UtN2FhODhjM2U4MDc0IiwgInNlcnZpY2VFbmRwb2ludCI6ICJodHRwOi8vYWdlbnQxLmxvY2FsaG9zdCIsICJsYWJlbCI6ICJNYWluIChhZG1pbikiLCAicmVjaXBpZW50S2V5cyI6IFsiQ3ZxdVdwN0NrcVdQU1FXTHFoeFRhQkFReGlmdU1XTlpDWWhSOG5FNTVxZXEiXX0=


In [119]:
agent2 = {}
endpoint = "http://localhost:8151/connections/create-admin-invitation-url"
adminInvitationResponse = requests.post(endpoint)
adminInvitationResponse = adminInvitationResponse.text
agent2['url'] = json.loads(adminInvitationResponse)['invitation_url']
print(agent2['url'])

http://agent2.localhost?c_i=eyJAdHlwZSI6ICJkaWQ6c292OkJ6Q2JzTlloTXJqSGlxWkRUVUFTSGc7c3BlYy9jb25uZWN0aW9ucy8xLjAvaW52aXRhdGlvbiIsICJAaWQiOiAiOGQxOTU4M2ItZTc0My00NWNjLWFkZjgtYjhmMTM1ZmZhOWZmIiwgInNlcnZpY2VFbmRwb2ludCI6ICJodHRwOi8vYWdlbnQyLmxvY2FsaG9zdCIsICJsYWJlbCI6ICJDbGllbnQgKGFkbWluKSIsICJyZWNpcGllbnRLZXlzIjogWyJKMjk0U2V1ZVA0dDFDdnMyeUJ3VG9nSnhybUttNjhRVWNVVEQyZ0pGVk5FaCJdfQ==


In [120]:
agent1 = util.processInviteUrl(agent1['url'])
agent2 = util.processInviteUrl(agent2['url'])
print("Decoded invitation:\n")
pp.pprint(agent2['invite'])

Decoded invitation:

{'@id': '8d19583b-e743-45cc-adf8-b8f135ffa9ff',
 '@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation',
 'label': 'Client (admin)',
 'recipientKeys': ['J294SeueP4t1Cvs2yBwTogJxrmKm68QUcUTD2gJFVNEh'],
 'serviceEndpoint': 'http://agent2.localhost'}


In [121]:
### Generating a did key using aries crypto module
# Create a key pair with random seed 
# Order: 0 - public key, 1 - secret / pritvate key
plugin['keypair'] = crypto.create_keypair()
# it seems to me that did is an encoded sub string of public key
# or maybe just some arbitrary random number
did = plugin['keypair'][0][:16]
# final version of our did
plugin['did'] = base58.b58encode(did).decode("ascii")
# encoding keys to base58
plugin['public_key_b58'] = \
    base58.b58encode(plugin['keypair'][0]).decode("ascii")
plugin['private_key_b58'] = \
    base58.b58encode(plugin['keypair'][1]).decode("ascii")

print("Private key: \n", plugin['private_key_b58'])
print("\nPublic key: \n", plugin['public_key_b58'])

Private key: 
 2miDPK4NNsN3paPpycuwK5bcTWKxaxwd3cECmS2GZkXrNc8MBD7CeFriD1w4LvDk9M3f5ewEoxxC1TCTUNYfhfwY

Public key: 
 22AohFsdKcE7GvonX8QJog19cADktAdz76HiRP1xzeiW


**Sending a connection request to acapy**

In [122]:
agent1['connection'] = util.connectWithAcapy(agent1, plugin)
agent2['connection'] = util.connectWithAcapy(agent2, plugin)
# print("Decoded acapy response: \n")
# pp.pprint(agent1['conn'])

**Unpacking connection data embedded in the response**

In [123]:
agent1['connection_details'] = util.decodeConnectionDetails(agent1['connection'])
agent2['connection_details'] = util.decodeConnectionDetails(agent2['connection'])

**Adding the connection to connection list**

In [124]:
agents = [
    {
        "label": agent1['invite']['label'],
        "DIDDoc": agent1['connection_details']['DIDDoc'],
        "myKey": plugin['keypair'][1]
    },
    {
        "label": agent2['invite']['label'],
        "DIDDoc": agent2['connection_details']['DIDDoc'],
        "myKey": plugin['keypair'][1]
    },
]

# pp.pprint(connections)

In [125]:
connection = util.connectAgents(agents[0], agents[1])
connection[1] = connection[1]['raw_repr']['connection_id']
connection[0] = connection[0]['connection_id']
print(connection[0], "\n", connection[1])

39ea7800-c4fb-47be-9ba5-4fa6b9d3af38 
 74b56774-33dc-41e4-8e7e-0447d40c0dcb


### FEATURE DISCOVERY

In [126]:
# pp.pprint(util.getFeatureDiscovery(agents[0]))

In [127]:
add = requests.post('http://localhost:8151/verifiable-services/add', json={
  "label": "memxsassdsssdfdeds",
  "service_schema": {
    "oca_schema_namespace": "string",
    "oca_schema_dri": "string"
  },
  "consent_schema": {
    "oca_schema_namespace": "string",
    "oca_schema_dri": "string",
    "data_url": "string"
  }
})
print(add.text)
add = json.loads(add.text)

{"created_at": "2020-08-18 12:26:29.343069Z", "consent_schema": {"oca_schema_dri": "string", "oca_schema_namespace": "string", "data_url": "string"}, "label": "memxsassdsssdfdeds", "updated_at": "2020-08-18 12:26:29.343069Z", "service_schema": {"oca_schema_dri": "string", "oca_schema_namespace": "string"}}


In [128]:
# did = requests.post('http://localhost:8151/wallet/did/create')
# did = json.loads(did.text)
# print(did)

In [129]:
# create_did = requests.post('http://localhost:8151/wallet/did/create')
# create_did = json.loads(create_did.text)
# print(create_did)

In [130]:
# register_did = requests.post('http://localhost:8151/ledger/register-nym', params={
#     "verkey": create_did['result']['verkey'],
#     "did": create_did['result']['did'],
# })
# register_did = json.loads(register_did.text)
# print(register_did)

In [131]:
# did = requests.post('http://localhost:8151/wallet/did/public', params={
#     'did': register_did['result']['did']
# })
# did = json.loads(did.text)
# print(did)

In [132]:
## DID

In [133]:
newDid1 = util.createDid(agents[1])
util.registerDidOnLedger(agents[1], newDid1['result']['did'], newDid1['result']['verkey'])

KeyboardInterrupt: 

In [134]:
newDid0 = util.createDid(agents[0])
util.registerDidOnLedger(agents[0], newDid0['result']['did'], newDid0['result']['verkey'])

KeyboardInterrupt: 

In [135]:
util.setPublicDid(agents[1], newDid1['result']['did'])
util.setPublicDid(agents[0], newDid0['result']['did'])

{'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/admin-dids/0.1/did',
 '@id': '77518c26-200e-4904-9f0c-2c103d9ab407',
 '~thread': {'thid': '8bc2c893-f0b5-4605-86d3-81a2fac12f4e'},
 'result': {'metadata': {'public': True},
  'verkey': '42JetKzj6MTUpbzh2jQuAK5h2m7MJcfwNswANsRTszXe',
  'did': '6Ye5hHnGx4ud1FK574urmk'}}

In [136]:
service_list = requests.get('http://localhost:8150/verifiable-services/request/' + connection[0])
service_list = json.loads(service_list.text)
pp.pprint(service_list)

{'connection_id': '39ea7800-c4fb-47be-9ba5-4fa6b9d3af38',
 'created_at': '2020-08-18 12:26:49.073804Z',
 'services': [{'consent_schema': {'data_url': 'string',
                                  'oca_schema_dri': 'string',
                                  'oca_schema_namespace': 'string'},
               'label': 'memxsassdsssdfdeds',
               'service_id': '0512c4da-640a-4205-adb7-973be1dc8009',
               'service_schema': {'oca_schema_dri': 'string',
                                  'oca_schema_namespace': 'string'}},
              {'consent_schema': {'data_url': 'string',
                                  'oca_schema_dri': 'string',
                                  'oca_schema_namespace': 'string'},
               'label': 'memxsassdsssdfdeds',
               'service_id': '5d361270-2ec5-4487-bd7d-a2aaf1d51153',
               'service_schema': {'oca_schema_dri': 'string',
                                  'oca_schema_namespace': 'string'}}],
 'updated_at': '2020-08-18 

In [137]:
apply = requests.post('http://localhost:8150/verifiable-services/apply', json={
  "connection_id": connection[0],
  "service_id": service_list['services'][0]['service_id'],
    "payload": "abc",
})
apply = json.loads(apply.text)
print(apply)

{'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/verifiable-services/1.0/application', '@id': 'dc7857f5-f679-4494-ba88-5f46d2052a8c', 'exchange_id': '91a5b0b5-2d84-4d90-8e05-63b889eaa02f', 'credential_definition_id': '6Ye5hHnGx4ud1FK574urmk:3:CL:454:consent_schema', 'data_dri': 'd0ef06ddb430ee91c55244cebb8ba590711c216929fd31f8f46261a74750e34d', 'service_id': '0512c4da-640a-4205-adb7-973be1dc8009'}


In [138]:
status = requests.get('http://localhost:8150/verifiable-services/apply-status', json={
  "connection_id": connection[0],
    "exchange_id": apply['exchange_id'],
})
status = json.loads(status.text)
pp.pprint(status)

[{'author': 'self',
  'connection_id': '39ea7800-c4fb-47be-9ba5-4fa6b9d3af38',
  'created_at': '2020-08-18 12:27:58.775785Z',
  'exchange_id': '91a5b0b5-2d84-4d90-8e05-63b889eaa02f',
  'label': 'memxsassdsssdfdeds',
  'service_id': '0512c4da-640a-4205-adb7-973be1dc8009',
  'state': 'pending',
  'updated_at': '2020-08-18 12:27:58.871237Z'}]


In [140]:
issue_self = requests.post('http://localhost:8151/verifiable-services/get-issue-self', json={
    "exchange_id": apply['exchange_id'],
})
issue_self = json.loads(issue_self.text)
pp.pprint(issue_self)

[{'author': 'other',
  'connection_id': '74b56774-33dc-41e4-8e7e-0447d40c0dcb',
  'consent_schema': '{"oca_schema_namespace": "string", "oca_schema_dri": '
                    '"string", "data_url": "string"}',
  'created_at': '2020-08-18 12:27:58.829836Z',
  'exchange_id': '91a5b0b5-2d84-4d90-8e05-63b889eaa02f',
  'issue_id': 'a68c1903d690b0724297e841db6d06b38ffac68b2941c05b5630ae1a203d6f43',
  'label': 'memxsassdsssdfdeds',
  'payload': 'abc',
  'service_id': '0512c4da-640a-4205-adb7-973be1dc8009',
  'service_schema': '{"oca_schema_namespace": "string", "oca_schema_dri": '
                    '"string"}',
  'state': 'pending',
  'updated_at': '2020-08-18 12:28:22.265858Z'}]


In [141]:
get_issue = requests.post('http://localhost:8151/verifiable-services/get-issue', json={
  "connection_id": connection[1],
    "exchange_id": apply['exchange_id'],
})
get_issue = json.loads(get_issue.text)
pp.pprint(get_issue)

{'@id': '456be761-cea6-4dca-af91-88ca8ee0ea7b',
 '@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/verifiable-services/1.0/get-issue',
 'exchange_id': '91a5b0b5-2d84-4d90-8e05-63b889eaa02f'}


In [99]:
issue_self = requests.post('http://localhost:8151/verifiable-services/process-application', json={
    "issue_id": issue_self[0]['issue_id'],
    "decision": "accept",
})
print(issue_self)
issue_self = json.loads(issue_self.text)
pp.pprint(issue_self)

<Response [200]>
{'credential_exchange_record': {'auto_issue': True,
                                'auto_offer': False,
                                'auto_remove': False,
                                'connection_id': 'c6c36e72-0a48-40e1-ab18-9c3217aac8a3',
                                'created_at': '2020-08-18 10:47:31.526225Z',
                                'credential_definition_id': 'FM3twPiVs8yTXUXkvVVixi:3:CL:449:Services',
                                'credential_exchange_id': '0dea803f-659d-4515-8175-599afe41349b',
                                'credential_offer': {'cred_def_id': 'FM3twPiVs8yTXUXkvVVixi:3:CL:449:Services',
                                                     'key_correctness_proof': {'c': '90995099090557260714165739600195560688067133351884776542462167511547636828367',
                                                                               'xr_cap': [['label',
                                                                               

In [36]:
payload = "some string yes dd"
payload_response = "response yay"
hash_id = hashlib.sha256(payload.encode("UTF-8")).hexdigest()
hash_id_response = hashlib.sha256(payload_response.encode("UTF-8")).hexdigest()
print(payload, hash_id)

some string yes dd b7c34035833ad394aaf721e431ae4a7f6359d66e1c69e8be9ec4c3e1d2e1ea1f


In [293]:
credentials = requests.get('http://localhost:8150/credentials')
credentials = json.loads(credentials.text) 
pp.pprint(credentials)

{'results': [{'attrs': {'consent_schema': '{"oca_schema_namespace": "string", '
                                          '"oca_schema_dri": "string", '
                                          '"data_url": "string"}',
                        'label': 'memxsassdsssdfdeds',
                        'service_schema': '{"oca_schema_namespace": "string", '
                                          '"oca_schema_dri": "string"}'},
              'cred_def_id': 'B6hy9V4FLjh6tQmknfEvTt:3:CL:273:Services',
              'cred_rev_id': None,
              'referent': 'e56cb0b5-f044-4176-b4d8-246c27ea0d00',
              'rev_reg_id': None,
              'schema_id': 'B6hy9V4FLjh6tQmknfEvTt:2:memxsassdsssdfdeds:1.0'},
             {'attrs': {'consent_schema': '{"oca_schema_namespace": "string", '
                                          '"oca_schema_dri": "string", '
                                          '"data_url": "string"}',
                        'label': 'memxsassdsssdfdeds',
         

I added returning hash id to add schema, I need to test if it works with both rejecting (not found schema) and accepting schema dound

### Add schema to storage

In [52]:
add = requests.post('http://localhost:8150/schema-storage/add', json={
  "payload": "test"
})
print(add.text)
add = json.loads(add.text)

{"hash_id": "9f86d081884c7d659a2feaa0c55ad015a3bf4f1b2b0b822cd15d6c15b0f00a08", "payload": "test"}


### SEND Initial Request

In [53]:
send = requests.post('http://localhost:8151/schema-exchange/request-schema', json={
  "hash_id": add['hash_id'],
  "connection_id": connection[1]
})

In [54]:
send = requests.post('http://localhost:8151/schema-exchange/request-schema', json={
  "hash_id": "ddd",
  "connection_id": connection[1]
})

In [55]:
print(send.text)

{"hash_id": "ddd", "connection_id": "a361d994-aaea-4f5c-9bf4-9644a774de2b", "record_id": "1e61b917-946b-4a56-8810-5b48081f79e6", "exchange_id": "c5f71769-b806-49fe-8861-15f49ffad9e3", "state": "pending"}


### GET Request By RECORD_ID

### GET Request List and filter by state and author

In [56]:
request_list = requests.post('http://localhost:8150/schema-storage/debug/request/list', json={})
request_list = json.loads(request_list.text)
print("Agent1:", )
pp.pprint(request_list)
request_list = requests.post('http://localhost:8151/schema-storage/debug/request/list', json={})
request_list = json.loads(request_list.text)
print("\nAgent2:")
pp.pprint(request_list)

Agent1:
[{'author': 'other',
  'connection_id': '844f6e7f-c13e-4ef8-b018-754fc5205445',
  'created_at': '2020-07-16 09:09:17.991960Z',
  'exchange_id': 'c5f71769-b806-49fe-8861-15f49ffad9e3',
  'payload': 'ddd',
  'state': 'failed to find the record',
  'updated_at': '2020-07-16 09:09:17.991960Z'},
 {'author': 'other',
  'connection_id': '844f6e7f-c13e-4ef8-b018-754fc5205445',
  'created_at': '2020-07-16 09:09:16.748526Z',
  'exchange_id': 'f51e4a06-2214-4b9e-a28a-d4c336258f66',
  'payload': '9f86d081884c7d659a2feaa0c55ad015a3bf4f1b2b0b822cd15d6c15b0f00a08',
  'state': 'approved',
  'updated_at': '2020-07-16 09:09:16.748526Z'}]

Agent2:
[{'author': 'self',
  'connection_id': 'a361d994-aaea-4f5c-9bf4-9644a774de2b',
  'created_at': '2020-07-16 09:09:17.950785Z',
  'exchange_id': 'c5f71769-b806-49fe-8861-15f49ffad9e3',
  'payload': 'ddd',
  'state': 'failed to find the record',
  'updated_at': '2020-07-16 09:09:18.029400Z'},
 {'author': 'self',
  'connection_id': 'a361d994-aaea-4f5c-9bf4-

### GET RECORD LIST

In [57]:
getRequests = requests.post('http://localhost:8150/schema-storage/list')
record_list = json.loads(getRequests.text)
print("Agent1: ")
pp.pprint(record_list)

getRequests = requests.post('http://localhost:8151/schema-storage/list')
record_list = json.loads(getRequests.text)
print("Agent2: ")
pp.pprint(record_list)

Agent1: 
[{'author': 'self',
  'created_at': '2020-07-16 09:09:15.407083Z',
  'hash_id': '9f86d081884c7d659a2feaa0c55ad015a3bf4f1b2b0b822cd15d6c15b0f00a08',
  'payload': 'test',
  'updated_at': '2020-07-16 09:09:15.407083Z'}]
Agent2: 
[{'author': 'a361d994-aaea-4f5c-9bf4-9644a774de2b',
  'created_at': '2020-07-16 09:09:16.808608Z',
  'hash_id': '9f86d081884c7d659a2feaa0c55ad015a3bf4f1b2b0b822cd15d6c15b0f00a08',
  'payload': 'test',
  'updated_at': '2020-07-16 09:09:16.808608Z'}]


In [ ]:

# message = util.buildMessage(
#         'https://didcomm.org/issue-credential/1.0/propose-credential'
#     )
# testing = util.sendMessage(message, agents[0])
# pp.pprint(testing)

In [ ]:
# pp.pprint(util.sendSchema(agents[0], "schesssdddvd", "1.0", ["memesssaaaaaaa"]))

In [ ]:
# result = util.schemaGetList(agents[0])
# last_schema_id = result['results'][-1]['schema_id']
# print(result)

In [ ]:
# util.schemaGet(agents[0], last_schema_id)